In [164]:
import pandas as pd
import os

In [165]:
os.chdir('/Users/jhbyun/Documents/685-Pr/ovarian-cancer-detection') #change directory

### 1. Import data

In [166]:
dwt4 = pd.read_csv(os.path.join(os.getcwd(), 'data/slope_chosenlevels_DWT_4302.csv'))
dwt8 = pd.read_csv(os.path.join(os.getcwd(), 'data/slope_chosenlevels_DWT_8702.csv'))

wp4 = pd.read_csv(os.path.join(os.getcwd(), 'data/slope_chosenlevels_WPD_Wang_4302.csv'))
wp8 = pd.read_csv(os.path.join(os.getcwd(), 'data/slope_chosenlevels_WPD_Wang_8702.csv'))

print(dwt4.shape)
print(dwt8.shape)

print(wp4.shape)
print(wp8.shape)

(199, 30)
(252, 30)
(199, 30)
(252, 30)


### 2. Label columns

In [167]:
col_names = [] #initialize empty list

for index, value in enumerate(dwt4.columns): #loop through number of columns in df
    col_name = "slope_" + str(index+1) #concat string with index
    col_names.append(col_name) #add to list
col_names.pop() #remove last element
col_names.append("state")

print(col_names)

dwt4.columns = col_names #set column names to slope_1, slope_2, ..., slope_29, state
dwt8.columns = col_names 

wp4.columns = col_names
wp8.columns = col_names

['slope_1', 'slope_2', 'slope_3', 'slope_4', 'slope_5', 'slope_6', 'slope_7', 'slope_8', 'slope_9', 'slope_10', 'slope_11', 'slope_12', 'slope_13', 'slope_14', 'slope_15', 'slope_16', 'slope_17', 'slope_18', 'slope_19', 'slope_20', 'slope_21', 'slope_22', 'slope_23', 'slope_24', 'slope_25', 'slope_26', 'slope_27', 'slope_28', 'slope_29', 'state']


In [168]:
%store dwt4
%store dwt8
%store wp4
%store wp8

Stored 'dwt4' (DataFrame)
Stored 'dwt8' (DataFrame)
Stored 'wp4' (DataFrame)
Stored 'wp8' (DataFrame)


### 3a. Fisher's criterion, dwt4

In [169]:
""" 
#sample of data to test melt function
samp = dwt4.iloc[95:105, :] #10 rows with both cases and controls
samp = samp[['slope_1', 'slope_2', 'slope_3', 'state']] #subset 3 features only for testing
samp = samp.reset_index(drop=True) 
samp

#pivot longer w/ melt function (https://pandas.pydata.org/docs/reference/api/pandas.melt.html#pandas.melt)
#results in 30 rows with each combination of state & slope 
samp_long = pd.melt(samp, id_vars=['state'], value_vars=['slope_1', 'slope_2', 'slope_3', 'state'])
samp_long 

samp_long.groupby(['state', 'variable']).mean()

#validate with manual slicing
samp_0 = samp[samp.state == 0]
samp_1 = samp[samp.state == 1]

print(samp_0)
print(samp_1)

print(samp_0.mean(axis=0))
print(samp_1.mean(axis=0)) 

"""

" \n#sample of data to test melt function\nsamp = dwt4.iloc[95:105, :] #10 rows with both cases and controls\nsamp = samp[['slope_1', 'slope_2', 'slope_3', 'state']] #subset 3 features only for testing\nsamp = samp.reset_index(drop=True) \nsamp\n\n#pivot longer w/ melt function (https://pandas.pydata.org/docs/reference/api/pandas.melt.html#pandas.melt)\n#results in 30 rows with each combination of state & slope \nsamp_long = pd.melt(samp, id_vars=['state'], value_vars=['slope_1', 'slope_2', 'slope_3', 'state'])\nsamp_long \n\nsamp_long.groupby(['state', 'variable']).mean()\n\n#validate with manual slicing\nsamp_0 = samp[samp.state == 0]\nsamp_1 = samp[samp.state == 1]\n\nprint(samp_0)\nprint(samp_1)\n\nprint(samp_0.mean(axis=0))\nprint(samp_1.mean(axis=0)) \n\n"

In [170]:
dwt4_long = pd.melt(dwt4, id_vars=['state'], value_vars=col_names)

dwt4_mean = dwt4_long.groupby(['state', 'variable'], as_index=False).mean() #mean for each window
dwt4_stdev = dwt4_long.groupby(['state', 'variable'], as_index=False).std() #stdev for each window

dwt4_summary = pd.merge(dwt4_mean, dwt4_stdev, how='inner', on=['state', 'variable']) #join mean and stdev
dwt4_summary.columns = ['state', 'slope', 'mean', 'stdev'] #rename columns

dwt4_summary = pd.pivot(dwt4_summary, columns='state', index='slope', values=['mean', 'stdev']) #pivot wider on state
dwt4_summary.columns = ['mean_0', 'mean_1', 'stdev_0', 'stdev_1'] #0 for controls, 1 for cases
dwt4_summary.head()

,mean_0,mean_1,stdev_0,stdev_1
slope,,,,
slope_1,-1.305751,-1.279610,0.111149,0.119663
slope_10,-1.839212,-1.479329,0.478302,0.377668
slope_11,-1.609969,-1.626537,0.440791,0.361923
slope_12,-1.775380,-1.754304,0.306839,0.268641
slope_13,-2.528880,-2.540485,0.144357,0.152308


In [171]:
dwt4_summary['fishers'] = (dwt4_summary['mean_1'] - dwt4_summary['mean_0'])**2 / (dwt4_summary['stdev_1']**2 + dwt4_summary['stdev_0']**2) #calculate Fisher's criterion for each window
dwt4_summary = dwt4_summary.sort_values(by='fishers', ascending=False)

dwt4_summary.reset_index(inplace=True)
dwt4_summary.head(n=10) #windows with ten highest Fisher's criterion values

,slope,mean_0,mean_1,stdev_0,stdev_1,fishers
0,slope_9,-1.880712,-1.518439,0.283776,0.242120,0.943158
1,slope_17,-2.435476,-1.967143,0.374610,0.572371,0.468725
2,slope_10,-1.839212,-1.479329,0.478302,0.377668,0.348718
3,slope_3,-0.865893,-1.020657,0.187331,0.187557,0.340856
4,slope_18,-2.271710,-2.075872,0.232540,0.345443,0.221173
5,slope_16,-1.700429,-1.894659,0.303630,0.289944,0.214034
6,slope_27,-1.918187,-2.129610,0.362621,0.300637,0.201463
7,slope_22,-2.410077,-2.568012,0.260068,0.239119,0.199848
8,slope_26,-1.951358,-2.187430,0.426593,0.311950,0.199539
9,slope_6,-1.783718,-1.674956,0.209041,0.202400,0.139718


In [172]:
five_windows = dwt4_summary.head(n=5)['slope'].to_list() #windows with 5 highest Fisher's criterion values
five_windows

['slope_9', 'slope_17', 'slope_10', 'slope_3', 'slope_18']

In [173]:
five_windows.append('state') #add state column
dwt4_five = dwt4[five_windows] #subset original dataframe with five windows with highest Fisher's criterion values
dwt4_five.head()

,slope_9,slope_17,slope_10,slope_3,slope_18,state
0,-1.374826,-2.518293,-1.101176,-0.968635,-2.663364,1
1,-1.381471,-2.239803,-0.835200,-1.182851,-2.181334,1
2,-1.764480,-1.346488,-1.673390,-1.044545,-1.373689,1
3,-1.316122,-2.524658,-0.873419,-1.074442,-2.283557,1
4,-1.590032,-2.356540,-1.582542,-1.065824,-2.259306,1


In [174]:
%store dwt4_five #store dataframe

Stored 'dwt4_five' (DataFrame)


UsageError: Unknown variable '#store'


### 3b. Fisher's criterion, dwt8

In [175]:
dwt8_long = pd.melt(dwt8, id_vars=['state'], value_vars=col_names)

dwt8_mean = dwt8_long.groupby(['state', 'variable'], as_index=False).mean() #mean for each window
dwt8_stdev = dwt8_long.groupby(['state', 'variable'], as_index=False).std() #stdev for each window

dwt8_summary = pd.merge(dwt8_mean, dwt8_stdev, how='inner', on=['state', 'variable']) #join mean and stdev
dwt8_summary.columns = ['state', 'slope', 'mean', 'stdev'] #rename columns

dwt8_summary = pd.pivot(dwt8_summary, columns='state', index='slope', values=['mean', 'stdev']) #pivot wider on state
dwt8_summary.columns = ['mean_0', 'mean_1', 'stdev_0', 'stdev_1'] #0 for controls, 1 for cases

dwt8_summary['fishers'] = (dwt8_summary['mean_1'] - dwt8_summary['mean_0'])**2 / (dwt8_summary['stdev_1']**2 + dwt8_summary['stdev_0']**2) #calculate Fisher's criterion for each window
dwt8_summary = dwt8_summary.sort_values(by='fishers', ascending=False)

dwt8_summary.reset_index(inplace=True)
five_windows = dwt8_summary.head(n=5)['slope'].to_list() #windows with 5 highest Fisher's criterion values

five_windows.append('state') #add state column
dwt8_five = dwt8[five_windows] #subset original dataframe with five windows with highest Fisher's criterion values
dwt8_five.head()


,slope_6,slope_29,slope_4,slope_11,slope_28,state
0,-2.664095,-2.344442,-2.322088,-2.428556,-2.701947,1
1,-2.848399,-2.603805,-2.477289,-2.089429,-2.744103,1
2,-2.738331,-2.401006,-2.373840,-1.990729,-2.614736,1
3,-2.718791,-2.421083,-2.333251,-1.712213,-2.682061,1
4,-2.749954,-2.458969,-2.245684,-2.332299,-2.854078,1


In [176]:
%store dwt8_five

Stored 'dwt8_five' (DataFrame)


### 3c. Fisher's criterion, wp4

In [177]:
wp4_long = pd.melt(wp4, id_vars=['state'], value_vars=col_names)

wp4_mean = wp4_long.groupby(['state', 'variable'], as_index=False).mean() #mean for each window
wp4_stdev = wp4_long.groupby(['state', 'variable'], as_index=False).std() #stdev for each window

wp4_summary = pd.merge(wp4_mean, wp4_stdev, how='inner', on=['state', 'variable']) #join mean and stdev
wp4_summary.columns = ['state', 'slope', 'mean', 'stdev'] #rename columns

wp4_summary = pd.pivot(wp4_summary, columns='state', index='slope', values=['mean', 'stdev']) #pivot wider on state
wp4_summary.columns = ['mean_0', 'mean_1', 'stdev_0', 'stdev_1'] #0 for controls, 1 for cases

wp4_summary['fishers'] = (wp4_summary['mean_1'] - wp4_summary['mean_0'])**2 / (wp4_summary['stdev_1']**2 + wp4_summary['stdev_0']**2) #calculate Fisher's criterion for each window
wp4_summary = wp4_summary.sort_values(by='fishers', ascending=False)

wp4_summary.reset_index(inplace=True)
five_windows = wp4_summary.head(n=5)['slope'].to_list() #windows with 5 highest Fisher's criterion values

five_windows.append('state') #add state column
wp4_five = wp4[five_windows] #subset original dataframe with five windows with highest Fisher's criterion values
wp4_five.head()

,slope_17,slope_18,slope_10,slope_9,slope_8,state
0,-1.533854,-1.657637,-0.491656,-0.924730,-1.003100,1
1,-1.197222,-1.186491,-0.348145,-0.820557,-1.093975,1
2,-0.480973,-0.577550,-0.899865,-1.336315,-1.401278,1
3,-1.458766,-1.291176,-0.354214,-1.113498,-1.109838,1
4,-1.288669,-1.234476,-0.789974,-1.281247,-1.428105,1


In [178]:
%store wp4_five

Stored 'wp4_five' (DataFrame)


### 3d. Fisher's criterion, wp8

In [179]:
wp8_long = pd.melt(wp8, id_vars=['state'], value_vars=col_names)

wp8_mean = wp8_long.groupby(['state', 'variable'], as_index=False).mean() #mean for each window
wp8_stdev = wp8_long.groupby(['state', 'variable'], as_index=False).std() #stdev for each window

wp8_summary = pd.merge(wp8_mean, wp8_stdev, how='inner', on=['state', 'variable']) #join mean and stdev
wp8_summary.columns = ['state', 'slope', 'mean', 'stdev'] #rename columns

wp8_summary = pd.pivot(wp8_summary, columns='state', index='slope', values=['mean', 'stdev']) #pivot wider on state
wp8_summary.columns = ['mean_0', 'mean_1', 'stdev_0', 'stdev_1'] #0 for controls, 1 for cases

wp8_summary['fishers'] = (wp8_summary['mean_1'] - wp8_summary['mean_0'])**2 / (wp8_summary['stdev_1']**2 + wp8_summary['stdev_0']**2) #calculate Fisher's criterion for each window
wp8_summary = wp8_summary.sort_values(by='fishers', ascending=False)

wp8_summary.reset_index(inplace=True)
five_windows = wp8_summary.head(n=5)['slope'].to_list() #windows with 5 highest Fisher's criterion values

five_windows.append('state') #add state column
wp8_five = wp8[five_windows] #subset original dataframe with five windows with highest Fisher's criterion values
wp8_five.head()

,slope_6,slope_27,slope_29,slope_17,slope_25,state
0,-1.787114,-1.857716,-1.443023,-1.771526,-1.781598,1
1,-1.935820,-1.848338,-1.619408,-1.778519,-1.853294,1
2,-1.857946,-1.857774,-1.516554,-1.685034,-1.783911,1
3,-1.834842,-1.861765,-1.478096,-1.663361,-1.813565,1
4,-1.863696,-1.877981,-1.488567,-1.515916,-1.817573,1


In [180]:
%store wp8_five

Stored 'wp8_five' (DataFrame)
